# Imports

In [1]:
import re
from unicodedata import normalize
import html
from tqdm.notebook import tqdm
import pickle
import gc
from itertools import compress
import spacy
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

# Load articles

In [2]:
data_artists = pickle.load(open("../Data/data_v3_persons.p", "rb"))
data_texts = pickle.load(open("../Data/data_v1_2.p", "rb"))

df_data_artists = pd.DataFrame(data_artists)
df_data_texts = pd.DataFrame(data_texts)


df_data_texts = df_data_texts.rename(columns={'text':'text_v2',
                                              'title':'_titulo_v2_',
                                              'url':'_url_'})

df = df_data_artists.merge(df_data_texts, 'left', left_on='_id_', right_on='id')

###

df = df.filter(['_id_', '_titulo_', '_url_', 'text', 'text_II', 'text_v2',
               'nombre_', 'ocupacion_',
               'nacimiento_', 'fallecimiento_', 'periodo_actividad_',
               'grupos_', 'instrumentos_', 'generos_', 'discograficas_',
               'relaciones_', 'tipo_voz_', 'religiones_', 'residencia_', 'premios_',
               'estudios_', 'idioma_', 'discografia_', 'singles_',
              ])

___

# Create dataset

In [3]:
def get_sello(row, periodo=False):

    try:
        if len(row)>0:
            sello = list()
            sello_periodo = list()
            
            for i,r in enumerate(row):
                sello.append('sello_'+str(i+1)+'::'+r['discografica'])
                
                if r['año_inicio']!='':
                    sello_periodo.append('sello periodo_'+str(i+1)+'::' + r['año_inicio'] + '-' + r['año_fin'])
                    
            if periodo:
                return sello_periodo
            else:
                return sello 
        
    except:
        return np.nan
    
####

df['sello'] = df.discograficas_.apply(get_sello)
df['sello periodo'] = df.discograficas_.apply(get_sello, periodo=True)       

In [4]:
def get_disco(row, año=False):

    try:
        if len(row)>0:
            disco = list()
            disco_año = list()
            
            for i,r in enumerate(row):
                disco.append('disco_'+str(i+1)+'::'+r['disco'])
                
                if r['año']!='':
                    disco_año.append('disco año_'+str(i+1)+'::'+r['año'])
                    
            if año:
                return disco_año
            else:
                return disco 
        
    except:
        return np.nan
    
####

df['disco'] = df.discografia_.apply(get_disco)
df['disco año'] = df.discografia_.apply(get_disco, año=True)

In [5]:
def get_single(row, año=False):

    try:
        if len(row)>0:
            single = list()
            single_año = list()
            
            for i,r in enumerate(row):
                single.append('single_'+str(i+1)+'::'+r['cancion'])
                
                if r['año']!='':
                    single_año.append('single año_'+str(i+1)+'::'+r['año'])
                    
            if año:
                return single_año
            else:
                return single 
        
    except:
        return np.nan
    
####

df['single'] = df.singles_.apply(get_single)
df['single año'] = df.singles_.apply(get_single, año=True)

In [6]:
def get_premio(row, año=False):

    try:
        if len(row)>0:
            premio = list()
            premio_año = list()
            
            for i,r in enumerate(row):
                premio.append('premio_'+str(i+1)+'::'+r['tipo'])
                
                if r['año']!='':
                    premio_año.append('premio año_'+str(i+1)+'::'+r['año'])
                    
            if año:
                return premio_año
            else:
                return premio 
        
    except:
        return np.nan
    
####

df['premio'] = df.premios_.apply(get_premio)
df['premio año'] = df.premios_.apply(get_premio, año=True)

In [7]:
def get_key_value(row, key):

    try:
        row = row[key]
        if len(row)>0:
            l = list()
            if isinstance(row, (str)):
                return row
            else:
                for r in row:
                    if r!='':
                        l.append(r)
                return l 
        
    except:
        return np.nan
    
####

df['nacimiento lugar'] = df.nacimiento_.apply(get_key_value, key='lugar')
df['nacimiento fecha'] = df.nacimiento_.apply(get_key_value, key='fecha')
df['nacionalidad'] = df.nacimiento_.apply(get_key_value, key='nacionalidad')

df['fallecimiento lugar'] = df.fallecimiento_.apply(get_key_value, key='lugar')
df['fallecimiento fecha'] = df.fallecimiento_.apply(get_key_value, key='fecha')
df['fallecimiento causa'] = df.fallecimiento_.apply(get_key_value, key='causa')

df['instrumento'] = df.instrumentos_.apply(get_key_value, key='tipo')
df['instrumento modelo'] = df.instrumentos_.apply(get_key_value, key='modelo')

df['nombre artistico'] = df.nombre_.apply(get_key_value, key='artistico')
df['nombre nacimiento'] = df.nombre_.apply(get_key_value, key='nacimiento')

In [8]:
def get_actividad(row):

    try:
        if len(row)>0:
            l = list()
            for r in row:
                if r['año_inicio']!='':
                    l.append(r['año_inicio'] + '-' + r['año_final'])
                    
            return l  
        
    except:
        return np.nan
    
####

df['actividad'] = df.periodo_actividad_.apply(get_actividad)

In [9]:
df['estudios'] = df['estudios_']
df['religion'] = df['religiones_']
df['idioma'] = df['idioma_']
df['genero'] = df['generos_']
df['tipo voz'] = df['tipo_voz_']
df['ocupacion'] = df['ocupacion_']
df['grupo'] = df['grupos_']
df['residencia'] = df['residencia_']

In [10]:
def get_relationship(row, relation, key='persona'):
    
    try:
        
        l_person = list()
        l_period = list()
        l_reason = list()
        
        row = [r for r in row if r['relación']==relation]
        
        if relation=='conyugue':
            
            for i,r in enumerate(row):
                l_person.append('conyugue_'+str(i+1)+'::'+r['persona'])
                
                if r['año_inicio']!='':
                    l_period.append('conyugue periodo_'+str(i+1)+'::'+r['año_inicio'] + '-' + r['año_final'])
                
                if r['razón_fin']!='':
                    l_reason.append('conyugue razon fin_'+str(i+1)+'::'+r['razón_fin'])
            
            
        elif relation=='pareja':
            
            for i,r in enumerate(row):
                
                l_person.append('pareja_'+str(i+1)+'::'+r['persona'])
                
                if r['año_inicio']!='':
                    l_period.append('pareja periodo_'+str(i+1)+'::'+r['año_inicio'] + '-' + r['año_final'])
                
        else:
            
            for i,r in enumerate(row): 
                l_person.append(r['persona'])

        
        if key=='persona':
            return l_person

        elif key=='periodo':
            return l_period

        elif key=='razon':
            return l_reason
            
    except:
        return np.nan
        

####

df['pareja'] = df.relaciones_.apply(get_relationship, relation='pareja', key='persona')
df['pareja periodo'] = df.relaciones_.apply(get_relationship, relation='pareja', key='periodo')

df['conyugue'] = df.relaciones_.apply(get_relationship, relation='conyugue', key='persona')
df['conyugue periodo'] = df.relaciones_.apply(get_relationship, relation='conyugue', key='periodo')
df['conyugue razon fin'] = df.relaciones_.apply(get_relationship, relation='conyugue', key='razon')

df['progenitor'] = df.relaciones_.apply(get_relationship, relation='progenitor', key='persona')
df['hijo/a'] = df.relaciones_.apply(get_relationship, relation='hijo', key='persona')
df['hermano/a'] = df.relaciones_.apply(get_relationship, relation='hermano', key='persona')

df['maestro/a'] = df.relaciones_.apply(get_relationship, relation='maestro', key='persona')
df['alumno/a'] = df.relaciones_.apply(get_relationship, relation='alumno', key='persona')

In [11]:
id_cols = ['_id_', '_titulo_', '_url_']

text_cols = [
        'text', 'text_II', 'text_v2', 
]


attr_cols = [
        'nombre artistico', 'nombre nacimiento',
        'ocupacion',
        'nacimiento lugar', 'nacimiento fecha', 'nacionalidad',
        'fallecimiento lugar', 'fallecimiento fecha', 'fallecimiento causa',
        'actividad',
        'residencia',
        'pareja', 'pareja periodo',
        'conyugue', 'conyugue periodo', 'conyugue razon fin',
        'progenitor', 'hijo/a', 'hermano/a',
        'grupo',
        'instrumento', 'instrumento modelo', 
        'premio', 'premio año',
        'tipo voz', 'idioma',
        'genero', 
        'sello', 'sello periodo',
        'disco', 'disco año',
        'single', 'single año', 
        'religion',  
        'estudios', 'maestro/a', 'alumno/a']

In [12]:
df = df.filter(id_cols+text_cols+attr_cols)

dataset = pd.melt(df, id_vars=id_cols+text_cols, value_vars=attr_cols)

# Clean dataset
dataset = dataset[dataset['value'].notna()]
dataset = dataset[dataset['value'].str.len() != 0]
dataset = dataset.explode('value')

In [13]:
for index in range(81,121)[::10]:
    
    print(index)
    print('---')
    print(dataset.iloc[index]['text'])
    print('-- -- -- ')
    print(dataset.iloc[index]['text_v2'])
    print('\n** ** **\n')

81
---
Gloria de los Ángeles Treviño Ruiz (Monterrey, 15 de febrero de 1968), conocida artísticamente como Gloria Trevi, es una cantante, actriz, productora y compositora mexicana.
-- -- -- 
Gloria de los Ángeles Treviño Ruiz (Monterrey, 15 de febrero de 1968), conocida artísticamente como Gloria Trevi, es una cantante, actriz, productora y compositora mexicana. 

** ** **

91
---
Anni-Frid Synni Lyngstad /ˈaniːfriːdˌsʏniː ˈlʏŋstɑːd/ (Ballangen, Narvik, Noruega), también conocida como Frida y por matrimonio Su Alteza Serenísima la princesa Anni-Frid de Reuss, condesa de Plauen, es una cantante y compositora sueca que forma parte del grupo ABBA.
-- -- -- 
Anni-Frid Synni Lyngstad /ˈaniːfriːdˌsʏniː ˈlʏŋstɑːd/ (Ballangen, Narvik, Noruega, ), también conocida como Frida y por matrimonio "Su Alteza Serenísima" la princesa Anni-Frid de Reuss, condesa de Plauen, es una cantante y compositora sueca que forma parte del grupo ABBA.

** ** **

101
---
Elis Regina Carvalho Costa, conocida popularm

In [14]:
dataset_target = dataset.filter(['_id_', 'text', 'text_II', 'text_v2'])
dataset_target = dataset_target.drop_duplicates()

## Clean text by length

In [15]:
def get_length(text: str):
    try:
        return len(text)
    except:
        return 0
    
dataset_target['length'] = dataset_target['text'].apply(get_length)
dataset_target['length_v2'] = dataset_target['text_v2'].apply(get_length)

dataset_target['length_diff'] = abs(dataset_target['length'] - dataset_target['length_v2'])

In [16]:
dataset_target.shape[0]

15430

In [17]:
equal_text = (dataset_target['length_diff']<15)
equal_text.sum()

12828

In [18]:
distinct_text = dataset_target[~equal_text]
distinct_text.shape[0]

2602

In [19]:
no_v2 = (dataset_target['length_v2']==0)
no_v2.sum()

1978

In [20]:
with_v2 = dataset_target[~no_v2]
with_v2.shape[0]

13452

In [21]:
just_II = ['10255', '221163', '228466', '250999', '254254', '276867', '300738', 
           '339247', '342738', '439857', '532906', '536168', '692313', '1008118',
           '1172571', '1366033', '1592822', '1678944', '1843858', '1888233',
           '2282334', '2998488', '3167394', '4262820', '4382979', '4641395',
           '4995825', '5026244', '6063030', '7387304', '7799838', '8004346',
           '8873852', '1458700', '1768426', '1386628', '2764984', '3803304',
           '128699', '645268', '2722328', '3208993', '3249319', '3335150', 
           '3973360', '5040083', '5516280', '7979272', '8581417',  ]

for j in just_II:
    dataset_target['text'] = np.where(dataset_target['_id_']==j,
                                      dataset_target['text_II'],
                                      dataset_target['text'])
    
    dataset_target['text_II'] = np.where(dataset_target['_id_']==j,
                                      '',dataset_target['text_II'])    

####

combine_I_and_II = ['122055', '213688', '921307', '1004060', '1252242',
                    '2206656', '2436111', '2488196', '2881530', '3750133',
                    '4892952', '4895423', '5013136', '5416951', '5576679',
                    '5848741', '6342499', '6403551', '6518953', '6577116',
                    '6824893', '6909202', '7330051', '7335553', '7336665',
                    '7537844', '7559444', '8234088', '8544497', '8730283',
                    '447972', '4786100', '4871790', '368877', '1371932',
                    '1364698', '1354770', '1344009', '1320410', '1318461',
                    '1289725', '1260773', '1120770', '1613988', '2094695',
                    '2190186', '2254983', '2321873', '2322613', '2400068',
                    '2403514', '2424325', '2424345', '2451316', '2459403',
                    '2470225', '2764370', '3045980', '3080841', '3183844',
                    '3249906', '3288430', '3288458', '3356740', '3571080',
                    '3821670', '3905448', '3909060', '4323309', '4342526',
                    '4349509', '4362455', '4359958', '4374278', '4377104', 
                    '4466307', '4801584', '4808064', '4838149', '5101120',
                    '5429015', '5440728', '5455716', '5488082', '5597506',
                    '5659679', '5721359', '5803060', '5827970', '6131343',
                    '6178244', '6523770', '6619162', '6632523', '6643696',
                    '6957676', '7038031', '7129838', '7331794', '7351014',
                    '7375846', '7826931', '8402744', '8559157', '3333360',
                    '7334132', '1572264', '1519',
                    ]

for c in combine_I_and_II:
    dataset_target['text'] = np.where(dataset_target['_id_']==c,
                                 np.where(dataset_target['text_II'].str[-1]=='.',
                                       dataset_target['text']+dataset_target['text_II'],
                                       dataset_target['text']+dataset_target['text_II'].str[0].str.lower()+dataset_target['text_II'].str[1:]),
                                       dataset_target['text'])
    
    dataset_target['text_II'] = np.where(dataset_target['_id_']==c,
                                      '',dataset_target['text_II'])  

####
    
v2 = ['430908', '1328669', '2985414', '3780017', '4363453', '4375538', '4816076',
      '4871987', '5233188', '5996913', '7282094', '7311356', '7315248', '7318540',
      '7391850', '7474329', '7826306', '8002030', '8305240', '8393838', '8047100',
      '7304516', '7509343', '8430136', ]

for v in v2:
    dataset_target['text'] = np.where(dataset_target['_id_']==v,
                                     dataset_target['text_v2'],
                                     dataset_target['text'])

In [22]:
dataset_target['length'] = dataset_target['text'].apply(get_length)
dataset_target['length_v2'] = dataset_target['text_v2'].apply(get_length)

### Split text into sentences

In [23]:
# Split into sentences

alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr|EE|UU|\Wn|ib)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"

def split_into_sentences(text:str, ind):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    
    if ind==0:
        if len(sentences)==0:
            print('-- -- --')
            print(text)
            return text
        else:
            return sentences[ind]
        
    elif ind==1:
        if len(sentences)<2:
            return ''
        else:
            return sentences[ind]
        
dataset_target['first_sentence'] = dataset_target['text'].apply(split_into_sentences, ind=0)
dataset_target['second_sentence'] = dataset_target['text'].apply(split_into_sentences, ind=1)

-- -- --
 José Plácido Domingo Embil (Madrid, 21 de enero de 1941), conocido como Plácido Domingo, es un cantante, director de orquesta, productor y compositor español,  
-- -- --
 María Concepción Alonso Bustillo (Cienfuegos, Cuba, 29 de junio de 1957), más conocida como María Conchita Alonso, es una actriz, cantante, compositora, exreina de belleza y filántropa venezolana, nacionalizada estadounidense en 2007  
-- -- --
 Ian MacKaye (pronunciado Mac-Ai), (16 de abril de 1962) es un músico estadounidense, conocido por ser el fundador y el dueño de  
-- -- --
 Óscar Sánchez Pérez (Sevilla, Andalucía, 24 de febrero de 1977), en sus inicios musicales conocido como DJ Blake y actualmente conocido como Acción Sánchez, es un productor  
-- -- --
 Christian «Flake» Lorenz (16 de noviembre de 1966 en Berlín Oriental) es un músico alemán, teclista de la banda de Neue Deutsche Härte Rammstein y antiguo integrante en la banda de punk rock Feeling B.  
-- -- --
 Curt Smith (nacido el 24 de junio 

-- -- --
 Solomon Popoli Linda (1909 - 8 de octubre 1962),  
-- -- --
 Raquel Lebrón, arpista paraguaya, ofreció numerosos conciertos y conferencias sobre las características del arpa paraguaya en Paraguay, Uruguay, Argentina, Brasil, Bolivia, Perú, Canadá, Sudáfrica, Medio Oriente, Asia y Europa, presentándose en los más importantes teatros y la televisión  
-- -- --
 Lize Marke, nacida Liliane Couck, (Wemmel, Bélgica, 1 de diciembre de 1936) es una cantante belga que es conocida por haber representado a Bélgica en el Festival de la Canción de Eurovisión 1965, con la canción "Als het weer lente is"  
-- -- --
 Carla Patricia Morrison Flores (Tecate, Baja California, 19 de julio de 1986), más conocida como Carla  
-- -- --
 Michael John David Westbrook (High Wycombe, Inglaterra, 21 de marzo de 1936) es un pianista, tubista, compositor, arreglista y director de orquesta de jazz inglés-  
-- -- --
 Chema Lara (Osuna, provincia de Sevilla, 20 de mayo de 1970) es un músico y periodista esp

-- -- --
   
-- -- --
 Augusto Osenda (n. Buenos Aires, Argentina, 23 de agosto de 1978) es un cantautor de Rock de Argentina, productor musical y miembro votante en La Academia Latina de la Grabación (  
-- -- --
 Edu Quindós, cantante, músico, compositor, actor y modelo publicitario español, conocido por sus dos álbumes en solitario Stay with me (2011), Hello (2013) y Still Believe (2016) y por ser vocalista y compositor de varios de los temas de la banda Plebeya  
-- -- --
 Alondra Santos (El Monte, California, 4 de febrero de 2002) es una cantante, actriz, y modelo mexicano-estadounidenses  
-- -- --
 Michael Richard Lewis (n. Pontypridd, nacido el 17 de agosto de 1975) es el guitarrista actual de No Devotion y ex-guitarristas de la banda Lostprophets y Public Disturbance  
-- -- --
 Marcia Ball, Orange (Texas), 20 de marzo de 1949, es una cantante y pianista de blues norteamericana  
-- -- --
 Ty Tabor (Pearl, Misisipi; 17 de septiembre de 1961) es un guitarrista estadounidense, p

In [24]:
# For testing
test = False

if test:
    for ind, row in dataset_target.iterrows():
        if ind<1200:
            continue
        if ind>1260:
            break
        else:
            print(row['_id_'])
            print(row['first_sentence'])
            print('----')
            print(row['second_sentence'])
            print('----')
            print('')
            print('****** ****** ******')

        if ind==1100:
            print(row['text'])
            print('- - - ')

In [25]:
dataset_target['full_text'] = (dataset_target['first_sentence']
                                            + ' '
                                            + dataset_target['second_sentence']).str.strip()

dataset_target = dataset_target.filter(['_id_', 'first_sentence', 'full_text'])
dataset = dataset.drop(columns=['text', 'text_II', 'text_v2'])
dataset = dataset.merge(dataset_target, 'left', '_id_')

In [26]:
dataset = dataset.drop_duplicates(['_id_', 'variable', 'value'])
dataset.shape

(219416, 7)

In [27]:
to_remove = ['6984853', '8578814', '1008118', '6984853', '8578814', '90526',
             '280108', '7329161', '4781182', '459452', '466539', '523917',
             '5415059', '8534109', '3747486', '6570541', '596855', '309896']
dataset = dataset[~dataset['_id_'].isin(to_remove)]
dataset.shape

(218383, 7)

In [28]:
dataset['variable2'] = dataset['variable']
dataset['variable2'] = np.where(dataset['variable2']=='nombre nacimiento',
                                     'nombre artistico', dataset['variable2'])

dataset = dataset.drop_duplicates(['_id_', 'variable2', 'value'])
dataset.drop(columns=['variable2'], inplace=True)
dataset.shape

(216518, 7)

## Filter by length

In [29]:
def get_length(text: str):
    try:
        return len(text)
    except:
        return 0

In [30]:
dataset['length'] = dataset['full_text'].apply(get_length)
dataset = dataset[dataset['length']<=512]
dataset.drop(columns=['length'], inplace=True)

## Drop some particular attributes

In [31]:
dataset[(dataset['value']=='Adryana Ribeiro')].head()

,_id_,_titulo_,_url_,variable,value,first_sentence,full_text
15327,5783805,Adryana Ribeiro,https://es.wikipedia.org/wiki?curid=5783805,nombre artistico,Adryana Ribeiro,Adryana Ribeiro es el sobrenombre de Adryana d...,Adryana Ribeiro es el sobrenombre de Adryana d...


In [32]:
variables_to_drop = ['sello', 'grupo']
mask = (dataset['_titulo_']=='Tech N9ne') & (dataset['variable'].isin(variables_to_drop))
dataset = dataset[~mask]

variables_to_drop = ['grupo']
mask = (dataset['_titulo_']=='Steve Aoki') & (dataset['variable'].isin(variables_to_drop))
dataset = dataset[~mask]
mask = (dataset['_titulo_']=='Marc Anthony') & (dataset['variable'].isin(variables_to_drop))
dataset = dataset[~mask]
mask = (dataset['_titulo_']=='David Guetta') & (dataset['variable'].isin(variables_to_drop))
dataset = dataset[~mask]
mask = (dataset['_titulo_']=='Snoop Dogg') & (dataset['variable'].isin(variables_to_drop))
dataset = dataset[~mask]
mask = (dataset['_titulo_']=='Miki González') & (dataset['variable'].isin(variables_to_drop))
dataset = dataset[~mask]
mask = (dataset['_titulo_']=='Young Thug') & (dataset['variable'].isin(variables_to_drop))
dataset = dataset[~mask]
mask = (dataset['_titulo_']=='Swizz Beatz') & (dataset['variable'].isin(variables_to_drop))
dataset = dataset[~mask]
mask = (dataset['_titulo_']=='Julieta Venegas') & (dataset['variable'].isin(variables_to_drop))
dataset = dataset[~mask]
mask = (dataset['_titulo_']=='Héctor Lavoe') & (dataset['variable'].isin(variables_to_drop))
dataset = dataset[~mask]
mask = (dataset['_titulo_']=='Gustavo Cerati') & (dataset['variable'].isin(variables_to_drop)) & (dataset['value']!='Soda Stereo')
dataset = dataset[~mask]
mask = (dataset['_titulo_']=='Enrique Mateu') & (dataset['variable'].isin(variables_to_drop))
dataset = dataset[~mask]
mask = (dataset['_titulo_']=='Wisin') & (dataset['variable'].isin(variables_to_drop))
dataset = dataset[~mask]
mask = (dataset['_titulo_']=='Diplo') & (dataset['variable'].isin(variables_to_drop))
dataset = dataset[~mask]
mask = (dataset['_titulo_']=='Phil Collins') & (dataset['variable'].isin(variables_to_drop))
dataset = dataset[~mask]
mask = (dataset['_titulo_']=='Maysa') & (dataset['variable'].isin(variables_to_drop))
dataset = dataset[~mask]
mask = (dataset['_titulo_']=='Adryana Ribeiro') & (dataset['variable'].isin(variables_to_drop))
dataset = dataset[~mask]

variables_to_drop = ['genero']
mask = (dataset['_titulo_']=='Fergie') & (dataset['variable'].isin(variables_to_drop))
dataset = dataset[~mask]

variables_to_drop = ['sello', 'sello periodo', 'genero']
mask = (dataset['_titulo_']=='Marion Elise Raven') & (dataset['variable'].isin(variables_to_drop))
dataset = dataset[~mask]

variables_to_drop = ['sello', 'sello periodo']
mask = (dataset['_titulo_']=='Brian Jones (músico)') & (dataset['variable'].isin(variables_to_drop))
dataset = dataset[~mask]

## Filter by # of attrributes

In [33]:
df_attr_per_artist = pd.DataFrame(dataset.groupby('_id_').count()['_titulo_']).rename(columns={'_titulo_': '#_attr'})

print(dataset.shape[0])
for i in range(1,4):
    list_id = df_attr_per_artist[df_attr_per_artist['#_attr']==i].reset_index()['_id_'].tolist()
    _id_list = dataset['_id_'].isin(list_id)
    dataset = dataset[~_id_list]
    print(dataset.shape[0])

212940
212895
212817
212592


## Save dataset

In [34]:
dataset_short = dataset.filter(['_id_', '_titulo_', '_url_', 'variable', 'value', 'first_sentence']).reset_index(drop=True)
dataset_short = dataset_short.rename(columns={'first_sentence': 'text'})
pickle.dump(dataset_short, open( "../Data/df_wikimusica_short.p", "wb"))

In [35]:
df_wikimusica_long = dataset.filter(['_id_', '_titulo_', '_url_', 'variable', 'value', 'full_text']).reset_index(drop=True)
df_wikimusica_long = df_wikimusica_long.rename(columns={'full_text': 'text'})
pickle.dump(df_wikimusica_long, open( "../Data/df_wikimusica_long.p", "wb"))